In [1]:
import os
import librosa
import gc
import ast
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import torchaudio
import sys
import datasets
from datasets import load_dataset,load_metric
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification, Trainer
from transformers import TrainingArguments

c:\Users\aagas\anaconda3\envs\finetuner\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 16

In [3]:
#load the dataset
dataset = load_dataset("audiofolder", data_dir="FoR Dataset Audio Detection\\for-norm\\for-norm")

In [4]:
#verify
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 53868
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 10798
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 4634
    })
})

In [5]:
#Check how it is loaded
dataset["test"][50]

{'audio': {'path': 'C:\\Users\\aagas\\Desktop\\Deepfake Audio_Detection\\FoR Dataset Audio Detection\\for-norm\\for-norm\\testing\\fake\\file1043.wav_16k.wav_norm.wav_mono.wav_silence.wav',
  'array': array([ 0.08981323,  0.08868408,  0.08905029, ...,  0.02703857,
         -0.00488281, -0.02178955]),
  'sampling_rate': 16000},
 'label': 0}

In [6]:
dataset["train"].features["label"]

ClassLabel(names=['fake', 'real'], id=None)

In [7]:
#label the real and fake
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [8]:
#verify the label
id2label

{'0': 'fake', '1': 'real'}

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

c:\Users\aagas\anaconda3\envs\finetuner\lib\site-packages\transformers\configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [10]:
max_duration = 5.0  # seconds

In [11]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )

    if "label" in examples:
        inputs["labels"] = [int(label) for label in examples["label"]]

    return inputs


In [12]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'labels'],
        num_rows: 53868
    })
    validation: Dataset({
        features: ['label', 'input_values', 'labels'],
        num_rows: 10798
    })
    test: Dataset({
        features: ['label', 'input_values', 'labels'],
        num_rows: 4634
    })
})

In [13]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [15]:
device

device(type='cuda')

In [16]:
#load_gpu
def to_device(data,device):
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking=True)

In [17]:
to_device(model,device)

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [18]:
model_name = model_checkpoint.split("/")[-1]


args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",

)

c:\Users\aagas\anaconda3\envs\finetuner\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
metric = load_metric("accuracy", trust_remote_code=True)

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


C:\Users\aagas\AppData\Local\Temp\ipykernel_21616\2033674808.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy", trust_remote_code=True)


In [20]:
class MyTrainer(Trainer):
    def training_step(self, model, inputs):
        if "labels" in inputs:
            inputs["labels"] = inputs["labels"].to(torch.int64)
        return super().training_step(model, inputs)


trainer = MyTrainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

  0%|          | 0/841 [00:00<?, ?it/s]c:\Users\aagas\anaconda3\envs\finetuner\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:862: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  1%|          | 10/841 [01:47<2:24:34, 10.44s/it]

{'loss': 0.6844, 'grad_norm': 1.5011638402938843, 'learning_rate': 3.5294117647058825e-06, 'epoch': 0.01}


  2%|▏         | 20/841 [03:47<2:43:43, 11.97s/it]

{'loss': 0.6539, 'grad_norm': 1.681717872619629, 'learning_rate': 7.058823529411765e-06, 'epoch': 0.02}


  4%|▎         | 30/841 [05:50<2:46:05, 12.29s/it]

...

{'loss': 0.0114, 'grad_norm': 0.018059974536299706, 'learning_rate': 4.365079365079365e-07, 'epoch': 0.99}


100%|█████████▉| 840/841 [2:17:00<00:09,  9.64s/it]

{'loss': 0.0008, 'grad_norm': 0.018556518480181694, 'learning_rate': 3.968253968253968e-08, 'epoch': 1.0}


100%|██████████| 841/841 [2:17:10<00:00,  9.74s/it]

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'

In [1]:
#Check
import os
import torch
import librosa
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

b:\Anaconda\envs\finetuner\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_checkpoint = "wav2vec2-base-finetuned-ks/checkpoint-841"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
model = AutoModelForAudioClassification.from_pretrained(model_checkpoint)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def process_audio(audio_file_path):
    # Load and resample the audio
    audio, sr = librosa.load(audio_file_path, sr=16000)

    # Extract features
    inputs = feature_extractor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    predicted_class_id = predictions.argmax().item()
    predicted_label = model.config.id2label[predicted_class_id]
    confidence = predictions[0][predicted_class_id].item()

    return predicted_label, confidence

# Specify the path to your audio file
audio_file_path = "AIVoiceGenerator_com_2-09-2024T14_0_51_ Salli.mp3"
# Process the audio file
predicted_label, confidence = process_audio(audio_file_path)

# Display results
print(f"Prediction: {predicted_label}")
print(f"Confidence: {confidence:.2f}")


Prediction: fake
Confidence: 1.00
